# Imports 

In [5]:
import dateutil
import json
import os
import requests
import pandas as pd
import numpy as np
import xlrd
from fbprophet import Prophet
import matplotlib.pyplot as plt

In [6]:
import datetime
from datetime import timedelta

In [7]:
%config Completer.use_jedi = False
%reload_ext lab_black

# Tracked bicyles in Berlin

In [8]:
xl = pd.ExcelFile("gesamtdatei_stundenwerte_2012-2019.xlsx", engine="openpyxl")
sheet_names = xl.sheet_names  # see all sheet names
list(enumerate(sheet_names))

[(0, 'Hinweise'),
 (1, 'Legende'),
 (2, 'Standortdaten'),
 (3, 'Jahresdatei 2012'),
 (4, 'Jahresdatei 2013'),
 (5, 'Jahresdatei 2014'),
 (6, 'Jahresdatei 2015'),
 (7, 'Jahresdatei 2016'),
 (8, 'Jahresdatei 2017'),
 (9, 'Jahresdatei 2018'),
 (10, 'Jahresdatei 2019')]

## Prepare data from excel into meaningful data frame

In [9]:
rs = []
for i in range(7, 10 + 1):
    tmpdf = xl.parse(sheet_names[i])
    tmpdf.columns = [c.split(" ")[0] for c in tmpdf.columns]
    tmpdf = tmpdf.rename({"Zählstelle": "dt"}, axis=1)
    tmpdf = tmpdf.melt(
        id_vars="dt",
        value_vars=tmpdf.columns.tolist()[1:],
        var_name="place",
        value_name="cnt",
    )
    tmpdf = tmpdf.dropna(subset=["dt"])
    tmpdf = tmpdf.fillna(0)
    tmpdf.set_index(["dt", "place"])
    rs.append(tmpdf)

In [10]:
df = pd.concat(rs)

In [11]:
place_mapping = (
    xl.parse(sheet_names[2]).dropna().rename({"Zählstelle": "place"}, axis=1)
)

In [12]:
df = df.merge(place_mapping, on="place")

In [13]:
df.columns = [
    "dt",
    "place",
    "cnt",
    "place_description",
    "lat",
    "lon",
    "dt_installation",
]

In [14]:
df[df.dt >= "2016-01-01"]

,dt,place,cnt,place_description,lat,lon,dt_installation
0,2016-01-01 00:00:00,02-MI-JAN-N,9.0,Jannowitzbrücke Nord,52.513932,13.417835,2015-04-01
1,2016-01-01 01:00:00,02-MI-JAN-N,26.0,Jannowitzbrücke Nord,52.513932,13.417835,2015-04-01
2,2016-01-01 02:00:00,02-MI-JAN-N,23.0,Jannowitzbrücke Nord,52.513932,13.417835,2015-04-01
3,2016-01-01 03:00:00,02-MI-JAN-N,28.0,Jannowitzbrücke Nord,52.513932,13.417835,2015-04-01
4,2016-01-01 04:00:00,02-MI-JAN-N,21.0,Jannowitzbrücke Nord,52.513932,13.417835,2015-04-01
...,...,...,...,...,...,...,...
841507,2019-12-31 19:00:00,27-RE-MAR,13.0,Markstraße,52.558190,13.364944,2015-05-01
841508,2019-12-31 20:00:00,27-RE-MAR,6.0,Markstraße,52.558190,13.364944,2015-05-01
841509,2019-12-31 21:00:00,27-RE-MAR,11.0,Markstraße,52.558190,13.364944,2015-05-01
841510,2019-12-31 22:00:00,27-RE-MAR,2.0,Markstraße,52.558190,13.364944,2015-05-01


## Add weather data

### Calculate costs for weather data

In [15]:
price_params = {"month": 12, "years": 4, "days": 31, "hours": 24, "price": 0.0001}
cost_calculation = f"$ {np.prod(list(price_params.values()))}"
cost_calculation

'$ 3.5712'

### chunking in terms of hourly is necessary, beacuse it's possible to collect 25k records in a request only

In [16]:
def collect_weather_data(key, lat, lon, start_dt, end_dt, hourly=False):
    dt_month_list = [
        s.strftime("%Y-%-m-%d")
        for s in list(
            dateutil.rrule.rrule(
                dateutil.rrule.MONTHLY,
                dtstart=datetime.date(*start_dt),
                until=datetime.date(*end_dt) + timedelta(days=30),
            )
        )
    ]
    weather_data = []
    for i in range(0, len(dt_month_list) - 1):
        print(i)
        tmp_start_dt = dt_month_list[i]
        tmp_end_dt = datetime.datetime.strftime(
            datetime.datetime.strptime(dt_month_list[i + 1], "%Y-%m-%d")
            - datetime.timedelta(days=1),
            "%Y-%-m-%d",
        )

        if hourly:
            url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lon}/{tmp_start_dt}/{tmp_end_dt}?unitGroup=metric&key={key}&include=obs,hours"
        else:
            url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lon}/{tmp_start_dt}/{tmp_end_dt}?unitGroup=metric&key={key}&include=obs"
        resp = requests.get(url)
        res_json = resp.json()
        for tmp in res_json["days"]:
            day = tmp["datetime"]
            if hourly:
                df_tmp = pd.DataFrame(tmp["hours"])
            else:
                df_tmp = pd.DataFrame(tmp)
            df_tmp.insert(loc=0, column="day", value=day)
            weather_data.append(df_tmp)
    df_weather = pd.concat(weather_data)
    return df_weather

### Apply chunked weather data collection

In [17]:
weather_api_params = dict(
    key=os.environ["WEATHER_DATA_SECRET"],
    lat="52.513932",
    lon="13.417835",
    start_dt=(2016, 1, 1),
    end_dt=(2020, 1, 1),
    hourly=False,
)

In [18]:
# df_weather = collect_weather_data(**weather_api_params)
# df_weather.to_csv("weather_data_daily.csv", sep=";", index=None)

In [19]:
df_weather = pd.read_csv("weather_data_daily.csv", sep=";", index_col=None)

In [20]:
df_weather["day"] = pd.to_datetime(df_weather["day"])
df_weather["hours"] = df_weather.apply(lambda x: int(str(x.datetime)[0:2]), axis=1)
df_weather["dt"] = df_weather.apply(
    lambda x: x.day + pd.Timedelta(hours=x.hours), axis=1
)

In [27]:
df_weather.head()

,day,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,sunsetEpoch,moonphase,conditions,description,icon,stations,source,tzoffset,hours,dt
0,2016-01-01,2016-01-01,1451602800,2.7,1.0,1.5,1.8,-1.5,0.2,1.2,...,1451660532,0.72,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,10382099999,obs,NaN,20,2016-01-01 20:00:00
1,2016-01-01,2016-01-01,1451602800,2.7,1.0,1.5,1.8,-1.5,0.2,1.2,...,1451660532,0.72,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,10385099999,obs,NaN,20,2016-01-01 20:00:00
2,2016-01-01,2016-01-01,1451602800,2.7,1.0,1.5,1.8,-1.5,0.2,1.2,...,1451660532,0.72,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,EDDB,obs,NaN,20,2016-01-01 20:00:00
3,2016-01-01,2016-01-01,1451602800,2.7,1.0,1.5,1.8,-1.5,0.2,1.2,...,1451660532,0.72,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,EDDT,obs,NaN,20,2016-01-01 20:00:00
4,2016-01-02,2016-01-02,1451689200,1.0,-7.1,-3.3,-1.4,-16.0,-9.6,-5.6,...,1451746997,0.77,Overcast,Cloudy skies throughout the day.,wind,10382099999,obs,NaN,20,2016-01-02 20:00:00


In [21]:
df_weather.columns

Index(['day', 'datetime', 'datetimeEpoch', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'precipcover', 'preciptype', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
       'sunrise', 'sunriseEpoch', 'sunset', 'sunsetEpoch', 'moonphase',
       'conditions', 'description', 'icon', 'stations', 'source', 'tzoffset',
       'hours', 'dt'],
      dtype='object')

In [22]:
df["day"] = df["dt"].dt.date

In [23]:
df_day_agg = df.groupby(["day", "place_description"], as_index=False)["cnt"].sum()
df_day_agg["day"] = pd.to_datetime(df_day_agg["day"])

In [24]:
df_weather_cleaned = (
    df_weather[["day", "tempmax", "tempmin", "humidity", "cloudcover", "windspeed"]]
    .drop_duplicates(subset="day")
    .fillna(0)
)
df_weather_cleaned.head(3)

,day,tempmax,tempmin,humidity,cloudcover,windspeed
0,2016-01-01,2.7,1.0,97.73,86.8,11.2
4,2016-01-02,1.0,-7.1,85.15,91.8,32.9
8,2016-01-03,-7.3,-11.3,74.78,44.2,29.8


In [25]:
dfm = df_day_agg.merge(df_weather_cleaned, on="day")
dfm = dfm.rename({"day": "dt"}, axis=1)
dfm.to_csv(
    "ts_bicycle_and_weather_berlin_day.csv.gz",
    sep="|",
    index=None,
    encoding="utf8",
    compression="gzip",
)

In [37]:
dfm[dfm.cnt > 0].head(10).reset_index(drop=True)

,dt,place_description,cnt,tempmax,tempmin,humidity,cloudcover,windspeed
0,2016-01-01,Alberichstraße,91.0,2.7,1.0,97.73,86.8,11.2
1,2016-01-01,Invalidenstraße Ost,233.0,2.7,1.0,97.73,86.8,11.2
2,2016-01-01,Invalidenstraße West,314.0,2.7,1.0,97.73,86.8,11.2
3,2016-01-01,Jannowitzbrücke Nord,530.0,2.7,1.0,97.73,86.8,11.2
4,2016-01-01,Jannowitzbrücke Süd,420.0,2.7,1.0,97.73,86.8,11.2
5,2016-01-01,Markstraße,165.0,2.7,1.0,97.73,86.8,11.2
6,2016-01-01,Monumentenstraße,566.0,2.7,1.0,97.73,86.8,11.2
7,2016-01-01,Oberbaumbrücke Ost,844.0,2.7,1.0,97.73,86.8,11.2
8,2016-01-01,Oberbaumbrücke West,776.0,2.7,1.0,97.73,86.8,11.2
9,2016-01-01,Paul-und-Paula-Uferweg,629.0,2.7,1.0,97.73,86.8,11.2


In [34]:
pd.read_csv("weather_data_hourly.csv", sep=";", index_col=None)

,day,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,icon,stations,source,tzoffset
0,2016-01-01,00:00:00,1451602800,1.7,0.3,94.79,1.0,NaN,NaN,0.0,...,3.4,90.4,NaN,NaN,0.0,Overcast,cloudy,"['10382099999', '10385099999', 'EDDB', 'EDDT']",obs,NaN
1,2016-01-01,01:00:00,1451606400,1.7,1.7,100.00,1.7,NaN,NaN,0.0,...,0.6,97.2,NaN,NaN,0.0,Overcast,fog,"['10382099999', '10385099999', 'EDDB', 'EDDT']",obs,NaN
2,2016-01-01,02:00:00,1451610000,2.6,1.1,93.97,1.7,NaN,NaN,0.0,...,0.5,64.0,NaN,NaN,0.0,Partially cloudy,fog,"['10382099999', '10385099999', 'EDDB', 'EDDT']",obs,NaN
3,2016-01-01,03:00:00,1451613600,2.7,1.1,94.83,2.0,NaN,NaN,0.0,...,1.3,97.2,NaN,NaN,0.0,Overcast,cloudy,"['10382099999', '10385099999', 'EDDB', 'EDDT']",obs,NaN
4,2016-01-01,04:00:00,1451617200,2.0,0.7,94.80,1.2,NaN,NaN,0.0,...,1.5,97.2,NaN,NaN,0.0,Overcast,cloudy,"['10382099999', '10385099999', 'EDDB', 'EDDT']",obs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2019-12-31,19:00:00,1577815200,5.4,2.0,77.60,1.8,0.0,NaN,0.0,...,42.2,NaN,NaN,NaN,0.0,Clear,clear-night,"['07389', '03376', '03987', '05825', '03015', ...",obs,NaN
35060,2019-12-31,20:00:00,1577818800,5.0,1.4,78.68,1.6,0.0,NaN,0.0,...,38.1,NaN,NaN,NaN,0.0,Clear,clear-night,"['07389', '03376', '03987', '05825', '03015', ...",obs,NaN
35061,2019-12-31,21:00:00,1577822400,4.1,0.3,81.01,1.1,0.0,NaN,0.0,...,34.1,NaN,NaN,NaN,0.0,Clear,clear-night,"['07389', '03376', '03987', '05825', '03015', ...",obs,NaN
35062,2019-12-31,22:00:00,1577826000,3.7,0.2,83.92,1.3,0.0,NaN,0.0,...,29.7,NaN,NaN,NaN,0.0,Clear,clear-night,"['07389', '03376', '03987', '05825', '03015', ...",obs,NaN
